## HTTP Servers and Clients

## Sending and receiving events with EventEmitters
EventEmitters are one of the core idioms of Node.js. Many of the core modules are EventEmitters, and also EventEmitters make an excellent skeleton to implement asynchronous programming. In this chapter, we'll work with the HTTPServer and HTTPClient objects. Both of them are subclasses of EventEmitter and rely on it to send events for each step of the HTTP protocol. 

In [1]:
// this defines a Pulser class

var events = require('events');
var util = require('util');

// Define the Pulser object
function Pulser() {
 events.EventEmitter.call(this);
}

util.inherits(Pulser, events.EventEmitter);

Pulser.prototype.start = function() {
 setInterval(() => {
 util.log('>>>> pulse');
 this.emit('pulse');
 util.log('<<<< pulse');
 }, 1000);
};

[Function]

This defines a Pulser class, which inherits from EventEmitter (using util.inherits). Its purpose is to send timed events, once a second, to any listeners. 

In [2]:
// Instantiate a Pulser object
var pulser = new Pulser();

// Handler function
pulser.on('pulse', () => {
 util.log('pulse received');
});

// Start it pulsing
pulser.start();

3 Jul 19:33:50 - >>>> pulse
3 Jul 19:33:50 - pulse received
3 Jul 19:33:50 - <<<< pulse
3 Jul 19:33:51 - >>>> pulse
3 Jul 19:33:51 - pulse received
3 Jul 19:33:51 - <<<< pulse
3 Jul 19:33:52 - >>>> pulse
3 Jul 19:33:52 - pulse received
3 Jul 19:33:52 - <<<< pulse
3 Jul 19:33:53 - >>>> pulse
3 Jul 19:33:53 - pulse received
3 Jul 19:33:53 - <<<< pulse
3 Jul 19:33:54 - >>>> pulse
3 Jul 19:33:54 - pulse received
3 Jul 19:33:54 - <<<< pulse
3 Jul 19:33:55 - >>>> pulse
3 Jul 19:33:55 - pulse received
3 Jul 19:33:55 - <<<< pulse


Here is an example from tutorials point...

In [1]:
// https://www.tutorialspoint.com/nodejs/nodejs_event_emitter.htm
    
var events = require('events');
var eventEmitter = new events.EventEmitter();

// listener #1
var listner1 = function listner1() {
   console.log('listner1 executed.');
}

// listener #2
var listner2 = function listner2() {
  console.log('listner2 executed.');
}

// Bind the connection event with the listner1 function
eventEmitter.addListener('connection', listner1);

// Bind the connection event with the listner2 function
eventEmitter.on('connection', listner2);

var eventListeners = require('events').EventEmitter.listenerCount(eventEmitter,'connection');
console.log(eventListeners + " Listner(s) listening to connection event");

// Fire the connection event 
eventEmitter.emit('connection');

// Remove the binding of listner1 function
eventEmitter.removeListener('connection', listner1);
console.log("Listner1 will not listen now.");

// Fire the connection event 
eventEmitter.emit('connection');

eventListeners = require('events').EventEmitter.listenerCount(eventEmitter,'connection');
console.log(eventListeners + " Listner(s) listening to connection event");

console.log("Program Ended.");

2 Listner(s) listening to connection event
listner1 executed.
listner2 executed.
Listner1 will not listen now.
listner2 executed.
1 Listner(s) listening to connection event
Program Ended.


The EventEmitter event has many names, and it can be anything that makes sense to you. You can define as many event names as you like. Event names are defined simply by calling .emit with the event name.

## HTTP server applications
In most cases, you'll be able to use an application framework such as Express that hides the HTTP protocol details, allowing the programmer to focus on business logic. We already saw a simple HTTP server application:

In [2]:
var http = require('http');
http.createServer((req, res) => {
 res.writeHead(200, {'Content-Type': 'text/plain'});
 res.end('Hello, World!\n');
}).listen(8124, '127.0.0.1');

console.log('Server running at http://127.0.0.1:8124');

Server running at http://127.0.0.1:8124


Now, let's look at something more interesting with different actions based on the URL.

In [3]:
var http = require('http');
var util = require('util');
var url = require('url');
var os = require('os');
var server = http.createServer();
server.on('request', (req, res) => {
 var requrl = url.parse(req.url, true);
 if (requrl.pathname === '/') {
 res.writeHead(200, {'Content-Type': 'text/html'});
 res.end(
`<html><head><title>Hello, world!</title></head>
<body><h1>Hello, world!</h1>
<p><a href='/osinfo'>OS Info</a></p>
</body></html>`);
 } else if (requrl.pathname === "/osinfo") {
 res.writeHead(200, {'Content-Type': 'text/html'});
 res.end(
`<html><head><title>Operating System Info</title></head>
<body><h1>Operating System Info</h1>
<table>
<tr><th>TMP Dir</th><td>${os.tmpDir()}</td></tr>
<tr><th>Host Name</th><td>${os.hostname()}</td></tr>
<tr><th>OS Type</th><td>${os.type()} ${os.platform()} ${os.arch()}
${os.release()}</td></tr>
<tr><th>Uptime</th><td>${os.uptime()} ${util.inspect(os.loadavg())}</
td></tr>
<tr><th>Memory</th><td>total: ${os.totalmem()} free: ${os.freemem()}</
td></tr>
<tr><th>CPU's</th><td><pre>${util.inspect(os.cpus())}</pre></td></tr>
<tr><th>Network</th><td><pre>${util.inspect(os.networkInterfaces())}</
pre></td></tr>
</table>
</body></html>`);
 } else {
 res.writeHead(404, {'Content-Type': 'text/plain'});
 res.end("bad URL "+ req.url);
 }
});
server.listen(8124);

console.log('listening to http://localhost:8124');

listening to http://localhost:8124


(node:11088) [DEP0022] DeprecationWarning: os.tmpDir() is deprecated. Use os.tmpdir() instead.


The real purpose of the template strings feature is supporting strings where we can easily substitute values directly into the string. Most other programming languages support this ability, and now JavaScript does too.